In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    "../data/train",
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary",
    subset="training"
)
val_generator = train_datagen.flow_from_directory(
    "../data/train",
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.


In [11]:
base_model = tf.keras.applications.ResNet50(
    input_shape=(150, 150, 3),
    include_top=False,
    weights="imagenet"
)

# Freeze base model
base_model.trainable = False

# Add custom classifier
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(train_generator, validation_data=val_generator, epochs=5)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 331s 3us/step


c:\Users\Mana\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 155s 718ms/step - accuracy: 0.5248 - loss: 0.7039 - val_accuracy: 0.6046 - val_loss: 0.6735
Epoch 2/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 150s 744ms/step - accuracy: 0.5628 - loss: 0.6785 - val_accuracy: 0.6052 - val_loss: 0.6660
Epoch 3/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 134s 666ms/step - accuracy: 0.5687 - loss: 0.6768 - val_accuracy: 0.6227 - val_loss: 0.6646
Epoch 4/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 113s 562ms/step - accuracy: 0.5778 - loss: 0.6713 - val_accuracy: 0.6021 - val_loss: 0.6629
Epoch 5/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 73s 364ms/step - accuracy: 0.5998 - loss: 0.6665 - val_accuracy: 0.6321 - val_loss: 0.6547


In [13]:
acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]
print(f"Train Accuracy: {acc:.2f}, Validation Accuracy: {val_acc:.2f}")

model.save("../models/cats_dogs_resnet.h5")

Train Accuracy: 0.60, Validation Accuracy: 0.63
